## Project: Search and Sample Return
### Writeup Template: You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---


**The goals / steps of this project are the following:**  

**Training / Calibration**  

* Download the simulator and take data in "Training Mode"
* Test out the functions in the Jupyter Notebook provided
* Add functions to detect obstacles and samples of interest (golden rocks)
* Fill in the `process_image()` function with the appropriate image processing steps (perspective transform, color threshold etc.) to get from raw images to a map.  The `output_image` you create in this step should demonstrate that your mapping pipeline works.
* Use `moviepy` to process the images in your saved dataset with the `process_image()` function.  Include the video you produce as part of your submission.

**Autonomous Navigation / Mapping**

* Fill in the `perception_step()` function within the `perception.py` script with the appropriate image processing functions to create a map and update `Rover()` data (similar to what you did with `process_image()` in the notebook). 
* Fill in the `decision_step()` function within the `decision.py` script with conditional statements that take into consideration the outputs of the `perception_step()` in deciding how to issue throttle, brake and steering commands. 
* Iterate on your perception and decision function until your rover does a reasonable (need to define metric) job of navigating and mapping.  

[//]: # (Image References)

[image1]: ./misc/rover_image.jpg
[image2]: ./calibration_images/example_grid1.jpg
[image3]: ./calibration_images/example_rock1.jpg 

## [Rubric](https://review.udacity.com/#!/rubrics/916/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  

You're reading it!

### Notebook Analysis
#### 1. Run the functions provided in the notebook on test images (first with the test data provided, next on data you have recorded). Add/modify functions to allow for color selection of obstacles and rock samples.
Here is an example of how to include an image in your writeup.

Description of the logic for sample and obstable identification.

Describe the logic in process_image()


![alt text][image1]

#### 1. Populate the `process_image()` function with the appropriate analysis steps to map pixels identifying navigable terrain, obstacles and rock samples into a worldmap.  Run `process_image()` on your test data using the `moviepy` functions provided to create video output of your result. 
And another! 

![alt text][image2]
### Autonomous Navigation and Mapping

#### 1. Fill in the `perception_step()` (at the bottom of the `perception.py` script) and `decision_step()` (in `decision.py`) functions in the autonomous mapping scripts and an explanation is provided in the writeup of how and why these functions were modified as they were.

Describe files and roles.
Describe core functionality with code for functionality.

show final result image.

#### 2. Launching in autonomous mode your rover can navigate and map autonomously.  Explain your results and how you might improve them in your writeup.  

**Note: running the simulator with different choices of resolution and graphics quality may produce different results, particularly on different machines!  Make a note of your simulator settings (resolution and graphics quality set on launch) and frames per second (FPS output to terminal by `drive_rover.py`) in your writeup when you submit the project so your reviewer can reproduce your results.**

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  



![alt text][image3]




## Project: Search and Sample Return
____

This implementation of the Search and Sample return project was ran on a Ubuntu 16.04 device with the following specifications:

    Memory: 7.7 GiB
    Processor: Intel Core i7-3537U CPU @ 2.00GHz x 4
    Graphics: Intel Ivybridge Mobile
    OS Type: 64-bit

The objective of the project was to provide a robot with perception and decision capability. The Robot should be able to do a mapping of its environment and optionally collect up to 6 rock sample.


### Notebook Analysis
___

#### Landmarks detection procedure:

##### Navigable Terrain:

Navigable terrain is detected from the warped field of vision of the robot, by thresholding and binarizing the vision image to filter out any pixel with RBG < (160, 160, 160).

##### Obstacles:

To locate the non-navigable terrain we reverse the procedure for navigable terrain and filter out all pixels with RGB > (160, 160, 160). Care must be taken to not return the black background in the image as obstacle. To do that a flag was added to the __color_thresh(image, rgb_thresh, invert, no_canvas)__ function to filter out the "canvas".

The function is provided below:
```python
def color_thresh(image, rgb_thresh=(160, 160, 160), invert=False, no_canvas=True):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(image[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (image[:,:,0] > rgb_thresh[0]) \
            & (image[:,:,1] > rgb_thresh[1]) \
            & (image[:,:,2] > rgb_thresh[2])
    if invert:
        above_thresh = ~above_thresh
    if no_canvas:
        is_not_canvas =  (image[:,:,0] != 0) \
            & (image[:,:,1]!= 0) \
            & (image[:,:,2] != 0)
        above_thresh = is_not_canvas & above_thresh
    # Index the array of zeros with the boolean array and set to 1
    # we add a guard against modifying the black background.
    color_select[above_thresh] = 1
    return color_select
```

##### Samples:

locating samples involves the following steps.
1. filtering the raw RGB image. To do so, we first apply a Gaussian Blur to the image, then run a two band threshold in HSV to isolate the yellow color and we binarize the result. This is performed by the function __rock_filter(image, hsv_lower, hsv_upper, ksize, sigmaX)__. A large kernel and variance is recommended to prevent some brightly lit obstacles to be false positive.
2. apply the perspective transform to the resulted image.
3. get the position of the best X,Y pixels to represent the center of the rock. Generate a circle around this point and return those as the potential rock position. This is performed by the function __find_rock(warped_rock, rock_radius)__

#### Overview of process_image():

This method collects all the detection functionality described above to generate the navigation view and the map for the Rover. The first step is the identification of navigable terrain, samples and obstacles by following the procedures detailed in the previous section. 

Once this is done, we use the telemetry of the Robot to check that it's Roll and pitch did not lead to poor readings from the camera. If Roll and Pitch are above a threshold of 1.5 degrees we do not send an update to the map.

The intensity of the pixels is updated gradually by increment of 10 with a cap at 255 and a floor at 0. This way as the Rover sees the same pixel repeatedly it increases it's confidence about the nature of the pixel.
To correct erroneous readings, we reduce the weight of a navigable pixel if it is identified as obstacle in the next reading. This allow for a self-correcting map that get cleaner as the Rover passes around the same areas. This is particularly useful to gain great certainty about the location of obstacles

The full procedure for map update is shown below:

```python
 if is_valide_image(roll, pitch , yaw, eroll=ERR_ROLL, epitch=ERR_PITCH):
        delta_update(data.worldmap, obstacle_y_world, obstacle_x_world, 0, DELTA)
        delta_update(data.worldmap, obstacle_y_world, obstacle_x_world, 2, -DELTA * 0.3)
        # update navigation
        delta_update(data.worldmap, nav_y_world, nav_x_world, 2, 2 * DELTA)
        # reduce certainty about obstacle.
        delta_update(data.worldmap, nav_y_world, nav_x_world, 0, -DELTA)        
        # update rock sample map
        delta_update(data.worldmap, rock_y_world, rock_x_world, 1, 255)
```

A drawback of this procedure is that the Rover is always as risk of forgetting some pieces of navigable terrain if it is in a narrow pass. A possible solution would be to lower the delta of the down-weighting based on a pixel's age. Pixels that have been set a long time ago should not need to be updated too aggressively

#### Additional work done in the notebook: 2D Cloud Point for wall navigation.

In addition to the points developed above, a section of the notebook focuses on building a simple form of cloud point. This is used as part of the Rover navigation system to help it find the best steering angle to crawl along a wall. 

A possible alternative usage of this feature would be obstacle avoidance, as we can identify the location of the obstacles around the Rover in its own coordinate frame, we should be able to devise a way for it to move away from then, akin to a potential field.

The relevant functions for this functionality are:

* __dominant_channel_filter(image, channel)__
* __world_to_pix(x_pix_world, y_pix_world, xpos, ypos, yaw,  scale)__
* __find_beam_points(worldmap, degrees, xpos, ypos, yaw, scale, atol)__

### Autonomous Navigation and Mapping
___
#### Structure of the project:

Several files were added to the project to unclutter the two files provided.
All the files and their content is described below:

- *image_processing.py* [new]: Provides a set of function to extract info from the image. most of the code for Rock and obstacle detection is stored there.

- *navigation.py* [new]: Provides a set of function to manipulate the map and vision with a focus on helping the Rover find its way. The code for the 2D point cloud is stored there.

- *perception.py* [from udacity]: Contains the code used to update the map and identify landmarks.

- *decision.py* [from udacity]: Provide the decision tree used by the Rover to navigate and the related functions.

- *drive_rover.py* [from udacity]: Provides Rover class and server to connect to the simulator. A new class was added to provide a stateful timer __DeltaTimer__. New attributes were added to the Rover to support its new functionalities. 

- *supporting_function.py* [from udacity]: Provide functions to update the Rover with new telemetry and update the maps for the simulator.



#### Overview of the decision tree:

The decision tree is broadly structured as follow:
```    
    [Has nav data?]
    Yes:
        [Is Stuck?]
         Yes:
            Mode = "stuck"
         No:
            [Sample in view?]
                 Yes:
                    move toward sample until close enough to pick it up.
                    Then transition to Mode = "stop" when sample has been picked up.
                No:
                    [Mode is forward?]
                        Do Forward
                    [Mode is stop?]
                        Do Stop
                    [Mode is stuck?]
                        Do Stuck
                   
    No:
        Adopt Default behavior.
```
        
#### Perception Step



#### Decision Step

#### Results and possible improvements: